In [45]:
from transformers import (
    AutoConfig, ElectraTokenizerFast, ElectraForTokenClassification, 
    DataCollatorForTokenClassification,
    TrainingArguments, Trainer,
)

from transformers.optimization import (
    AdamW, get_linear_schedule_with_warmup,
    Adafactor, AdafactorSchedule,
)

import torch
import wandb

import datasets
import evaluate

from sklearn.metrics import accuracy_score, f1_score

import numpy as np
import pandas as pd

import os
import re
import random

import demoji

In [46]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [47]:
model_checkpoint = 'training_results/monologg_koelectra_base_v3_discriminator_uncleaned_v11/target_tagger/monologg_koelectra_base_v3_discriminator_uncleaned_v11/checkpoint-370'

tokenizer = ElectraTokenizerFast.from_pretrained(model_checkpoint)
model = ElectraForTokenClassification.from_pretrained(model_checkpoint).to(device)

In [48]:
# def preprocess_function(examples):
#     input_ids = tokenizer.convert_tokens_to_ids(examples["input_tokens_list"])
#     token_type_ids = [0 for _ in range(len(input_ids))]
#     attention_mask = [1 for _ in range(len(input_ids))]   
    
#     input_triplet = dict(
#         input_ids = torch.tensor([input_ids]).to(device),
#         token_type_ids = torch.tensor([token_type_ids]).to(device),
#         attention_mask = torch.tensor([attention_mask]).to(device),
#     )
#     return input_triplet

In [49]:
TRAIN_DATA_PATH = './dataset/uncleaned_v11/tagger_train.json'
EVAL_DATA_PATH = './dataset/uncleaned_v11/tagger_dev.json'

train_dataset = pd.read_json(TRAIN_DATA_PATH)
eval_dataset = pd.read_json(EVAL_DATA_PATH)

In [50]:
total = pd.concat([train_dataset, eval_dataset])

In [51]:
# for idx, row in total.iterrows():
#     for annotation in row.annotation:
#         if annotation[1][0] == '':
#             print(row.id)

nikluge-sa-2022-train-01010
nikluge-sa-2022-dev-00515
nikluge-sa-2022-train-00599
nikluge-sa-2022-dev-02117
nikluge-sa-2022-train-00123
nikluge-sa-2022-dev-01401
nikluge-sa-2022-dev-02778
nikluge-sa-2022-dev-02261
nikluge-sa-2022-train-00619
nikluge-sa-2022-dev-02269
nikluge-sa-2022-train-00581
nikluge-sa-2022-train-01564
nikluge-sa-2022-train-02511
nikluge-sa-2022-dev-02630
nikluge-sa-2022-train-01040
nikluge-sa-2022-dev-00496
nikluge-sa-2022-dev-01330
nikluge-sa-2022-train-01305
nikluge-sa-2022-train-00265
nikluge-sa-2022-dev-00499
nikluge-sa-2022-dev-00888
nikluge-sa-2022-dev-02411
nikluge-sa-2022-train-01286
nikluge-sa-2022-dev-00889
nikluge-sa-2022-dev-02365
nikluge-sa-2022-train-02412
nikluge-sa-2022-train-00954
nikluge-sa-2022-dev-01061
nikluge-sa-2022-train-02014
nikluge-sa-2022-dev-02524


In [52]:
wrong = ['nikluge-sa-2022-train-00265',
'nikluge-sa-2022-train-00619',
'nikluge-sa-2022-train-01286',
'nikluge-sa-2022-train-02412',
'nikluge-sa-2022-train-02511',
'nikluge-sa-2022-dev-00515',
'nikluge-sa-2022-dev-01061',
'nikluge-sa-2022-dev-02365']

In [53]:
for idx, row in total[total.id.isin(wrong)].iterrows():
    print(row.sentence_form)
    print(row.annotation)
    print()

Target 피부샵 전문가들도 사용하는 탁월한 피부관리 마스크팩인 스킨덤 디데이팩 5스텝 탄력 마스크팩
[['제품 전체#인지도', ['', 37, 36], 'positive']]

Target 피부 상황에 따라 다르게 앰플을 골라 사용 할 수 있는 제품인 지디일레븐 셀 팩토리 주사기 앰플
[['본품#다양성', ['', 42, 24], 'positive']]

Target 매일 &name&의 몸에 직접 닿는 아기바디워시인지라 순하면서 자극없고 안전한 성분을 사용해야겠다 생각이 들어서 저자극으로 유해성분이 없는 아기바디워시를 찾다가 에바비바 바디워시를 선택했어요
[['본품#품질', ['', 97, 36], 'positive']]

Target 요즘처럼 피부고민 많을 때 푸석거리는 피부관리에 시트마스크팩 찾고 계시면 프로뮨 마스크팩 한번 써보시길요!
[['제품 전체#일반', ['', 48, 40], 'positive']]

Target 마스크는 호흡은 편해야 하고 차단은 강해야 하는데 리인데이 에이스 미세먼지 마스크 추천드려요!!
[['제품 전체#품질', ['', 35, 11], 'neutral']]

Target &name&이 사용하는 콜라겐마스크팩 토기가 사용해본 어바웃미 더바브랜드의 메디앤서 콜라겐 퍼밍업 마스크 !
[['제품 전체#인지도', ['', 37, 27], 'positive']]

Target 거뭇거뭇 한 내 피부를 위해 사용중인 겟잇뷰티&name&미백크림 으로 핫한 블랙샷프리미엄 미백크림
[['제품 전체#인지도', ['', 49, 42], 'positive']]

Target 4구 짜리 멀티탭을 사용하다보면 선들때문에 지저분해 보였는데 아이디바 멀티탭 사용하니 정말 깨끗해진 느낌이라 좋아요ㅎㅎ
[['제품 전체#일반', ['', 41, 17], 'positive']]



In [54]:
train_json = './dataset/nikluge-sa-2022-train.jsonl'
dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
test_json = './dataset/nikluge-sa-2022-test.jsonl'

train = pd.read_json(train_json, lines=True)
dev = pd.read_json(dev_json, lines=True)
test = pd.read_json(test_json, lines=True)

train = train.drop(2319)
dev = dev.drop(1692)

ori_total = pd.concat([train, dev])

In [59]:
# for idx, row in ori_total[ori_total.id.isin(wrong)].iterrows():
#     print(row.sentence_form)
#     print(row.annotation)
#     print()

요즘처럼 피부고민 많을 때 푸석거리는 피부관리에 #시트마스크팩 찾고 계시면 프로뮨 마스크팩 한번 써보시길요!
[['제품 전체#일반', ['프로뮨 마스크팩', 42, 50], 'positive']]

피부 상황에 따라 다르게 앰플을 골라 사용 할 수 있는 제품인 지디일레븐 셀 팩토리 주사기 앰플
[['본품#다양성', ['지디일레븐 셀 팩토리 주사기 앰플', 35, 53], 'positive']]

마스크는 호흡은 편해야 하고 차단은 강해야 하는데 리인데이 에이스 미세먼지 마스크 추천드려요!!
[['제품 전체#품질', ['리인데이 에이스 미세먼지 마스크', 28, 45], 'neutral']]

거뭇거뭇 한 내 피부를 위해 사용중인 #겟잇뷰티&name&미백크림 으로 핫한 #블랙샷프리미엄 #미백크림
[['제품 전체#인지도', ['블랙샷프리미엄 #미백크림', 44, 57], 'positive']]

매일 &name&의 몸에 직접 닿는 아기바디워시인지라 순하면서 자극없고 안전한 성분을 사용해야겠다 생각이 들어서 저자극으로 유해성분이 없는 아기바디워시를 찾다가 에바비바 바디워시를 선택했어요
[['본품#품질', ['에바비바 바디워시', 90, 99], 'positive']]

피부샵 전문가들도 사용하는 탁월한 피부관리 마스크팩인 스킨덤 디데이팩 5스텝 탄력 마스크팩
[['제품 전체#인지도', ['스킨덤 디데이팩 5스텝 탄력 마스크팩', 30, 50], 'positive']]

#4구 짜리 멀티탭을 사용하다보면 선들때문에 지저분해 보였는데 #아이디바 #멀티탭 사용하니 정말 깨끗해진 느낌이라 좋아요ㅎㅎ
[['제품 전체#일반', ['아이디바 #멀티탭', 36, 45], 'positive']]

&name&이 사용하는 콜라겐마스크팩 토기가 사용해본 어바웃미 더바브랜드의 메디앤서 콜라겐 퍼밍업 마스크 !
[['제품 전체#인지도', ['어바웃미 더바브랜드의 메디앤서 콜라겐 퍼밍업 마스크', 30, 58], 'positive']]



In [ ]:
# 요즘처럼 피부고민 많을 때 푸석거리는 피부관리에 #시트마스크팩 찾고 계시면 프로뮨 마스크팩 한번 써보시길요!
# [['제품 전체#일반', ['프로뮨 마스크팩', 42, 50], 'positive']]

# 피부 상황에 따라 다르게 앰플을 골라 사용 할 수 있는 제품인 지디일레븐 셀 팩토리 주사기 앰플
# [['본품#다양성', ['지디일레븐 셀 팩토리 주사기 앰플', 35, 53], 'positive']]

# 마스크는 호흡은 편해야 하고 차단은 강해야 하는데 리인데이 에이스 미세먼지 마스크 추천드려요!!
# [['제품 전체#품질', ['리인데이 에이스 미세먼지 마스크', 28, 45], 'neutral']]

# 거뭇거뭇 한 내 피부를 위해 사용중인 #겟잇뷰티&name&미백크림 으로 핫한 #블랙샷프리미엄 #미백크림
# [['제품 전체#인지도', ['블랙샷프리미엄 #미백크림', 44, 57], 'positive']]

# 매일 &name&의 몸에 직접 닿는 아기바디워시인지라 순하면서 자극없고 안전한 성분을 사용해야겠다 생각이 들어서 저자극으로 유해성분이 없는 아기바디워시를 찾다가 에바비바 바디워시를 선택했어요
# [['본품#품질', ['에바비바 바디워시', 90, 99], 'positive']]

# 피부샵 전문가들도 사용하는 탁월한 피부관리 마스크팩인 스킨덤 디데이팩 5스텝 탄력 마스크팩
# [['제품 전체#인지도', ['스킨덤 디데이팩 5스텝 탄력 마스크팩', 30, 50], 'positive']]

# #4구 짜리 멀티탭을 사용하다보면 선들때문에 지저분해 보였는데 #아이디바 #멀티탭 사용하니 정말 깨끗해진 느낌이라 좋아요ㅎㅎ
# [['제품 전체#일반', ['아이디바 #멀티탭', 36, 45], 'positive']]

# &name&이 사용하는 콜라겐마스크팩 토기가 사용해본 어바웃미 더바브랜드의 메디앤서 콜라겐 퍼밍업 마스크 !
# [['제품 전체#인지도', ['어바웃미 더바브랜드의 메디앤서 콜라겐 퍼밍업 마스크', 30, 58], 'positive']]

In [6]:
# while True:
#     k = random.randrange(len(train_dataset))
#     input_tokens_list = train_dataset['input_tokens_list'][k]
#     sentence_form = train_dataset['sentence_form'][k]
#     annotations = train_dataset['annotation'][k]
#     if len(tokenizer.convert_tokens_to_ids(input_tokens_list)) != len(tokenizer.encode(sentence_form)):
#         print(k)
#         print(len(tokenizer.convert_tokens_to_ids(input_tokens_list)), len(tokenizer.encode(sentence_form)))
#         print(sentence_form)
#         print(annotations)
#         break

In [7]:
idx = random.randrange(len(eval_dataset))
sample = eval_dataset.iloc[idx]
# sample = train_dataset.iloc[1151]
print('original: ')
print(sample.annotation)
print(sample.sentence_form)
print()

input_triplet = tokenizer(sample.sentence_form, return_tensors='pt')
input_triplet = {k:v.to(device) for k, v in input_triplet.items()}
# input_triplet = preprocess_function(sample)
output = model(**input_triplet).logits

print('label:'), print(sample.labels[1:-1])
print('prediction:'), print(output.argmax(-1)[-1].tolist()[1:-1])
print()
print(output.argmax(-1)[-1].tolist()[1:-1] == sample.labels[1:-1])
print()
toks = sample.input_tokens_list[1:-1]
pred = output.argmax(-1)[-1].tolist()[1:-1]
# labs = sample.labels

starts = list({k:v for k, v in enumerate(pred) if v == 1}.keys())
targets = []
for start in starts:
    target = [toks[start]]
    for tok, lab in zip(toks[start+1:], pred[start+1:]):
        if lab != 2:
            break
        else:
            target.append(tok)
    targets.append(tokenizer.convert_tokens_to_string(target))

print('targets in labels:')
for el in sample.annotation:
    print(el[1][0])
print('targets predicted:')
print(targets)

original: 
[['제품 전체#일반', ['스웨터', 30, 33], 'positive']]
Target 오늘입고 옷장속으로 들어갈 내가 좋아하는 스웨터

label:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
prediction:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

True

targets in labels:
스웨터
targets predicted:
['스웨터']


In [8]:
tokenizer.encode('살랑드파리 , 아쿠아컴플릿시리즈') == tokenizer.encode('살랑드파리, 아쿠아컴플릿시리즈')

True

In [78]:
idx = random.randrange(len(eval_dataset))
sample = eval_dataset.iloc[idx]
# sample = train_dataset.iloc[1151]
print('original: ')
print(sample.annotation)
print(sample.sentence_form)
print()

original: 
[['본품#품질', ['달바 미스트 세럼..!', 30, 42], 'positive']]
Target 촤르르~~피부 윤광 장난아니게 만들어주는 달바 미스트 세럼..!



In [79]:
sentence = sample.sentence_form
sentence = re.sub('#', '', sentence)
sentence = re.sub('\xa0', ' ', sentence)
print(sentence)

Target 촤르르~~피부 윤광 장난아니게 만들어주는 달바 미스트 세럼..!


In [80]:
tokens = tokenizer.tokenize(sentence)
ids = tokenizer.encode(sentence)[1:-1]
print(tokens == tokenizer.convert_ids_to_tokens(ids))
print(tokens)
print(tokenizer.convert_ids_to_tokens(ids))

True
['T', '##ar', '##ge', '##t', '촤', '##르르', '~', '~', '피부', '윤', '##광', '장난', '##아', '##니', '##게', '만들', '##어', '##주', '##는', '달', '##바', '미스트', '세', '##럼', '.', '.', '!']
['T', '##ar', '##ge', '##t', '촤', '##르르', '~', '~', '피부', '윤', '##광', '장난', '##아', '##니', '##게', '만들', '##어', '##주', '##는', '달', '##바', '미스트', '세', '##럼', '.', '.', '!']


In [81]:
input_triplet = tokenizer(sentence, return_tensors='pt')
input_triplet = {k:v.to(device) for k, v in input_triplet.items()}
output = model(**input_triplet).logits
pred = output.argmax(-1)[-1].tolist()[1:-1]
print(pred)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0]


In [82]:
starts = list({k:v for k, v in enumerate(pred) if v == 1}.keys())
targets = []
for start in starts:
    target = [tokens[start]]
    for tok, lab in zip(tokens[start+1:], pred[start+1:]):
        if lab != 2:
            break
        else:
            target.append(tok)
    targets.append(tokenizer.convert_tokens_to_string(target))

print('targets predicted:')
print(targets)

targets predicted:
['달바 미스트 세럼']


In [83]:
sentence = sample.sentence_form
sentence = re.sub('#', '', sentence)
sentence = re.sub('\xa0', ' ', sentence)

tokens = tokenizer.tokenize(sentence)
ids = tokenizer.encode(sentence)[1:-1]

input_triplet = tokenizer(sentence, return_tensors='pt')
input_triplet = {k:v.to(device) for k, v in input_triplet.items()}

output = model(**input_triplet).logits
pred = output.argmax(-1)[-1].tolist()[1:-1]

starts = list({k:v for k, v in enumerate(pred) if v == 1}.keys())
targets = []
for start in starts:
    target = [tokens[start]]
    for tok, lab in zip(tokens[start+1:], pred[start+1:]):
        if lab != 2:
            break
        else:
            target.append(tok)
    targets.append(tokenizer.convert_tokens_to_string(target))

print('targets predicted:')
print(targets)

targets predicted:
['달바 미스트 세럼']
